**Importing the dependencies**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.model_selection import train_test_split

**Data collection and processing**

In [ ]:
# loading the data from sklearn
breast_cancer_dataset = sklearn.datasets.load_breast_cancer()

In [ ]:
print(breast_cancer_dataset)

In [ ]:
# loading the data to a data frame
data_frame = pd.DataFrame(breast_cancer_dataset.data, columns = breast_cancer_dataset.feature_names)

In [ ]:
# print the first 5 rows of the dataframe
data_frame.head()

In [ ]:
# adding the 'target' column to the data frame
data_frame['label'] = breast_cancer_dataset.target

In [ ]:
# print the last 5 rows of the dataframe
data_frame.tail()

In [ ]:
# prints the number of rows and columns in the dataset
data_frame.shape

In [ ]:
# getting some information about this data
data_frame.info()

In [ ]:
# checking for missing values
data_frame.isnull().sum()

In [ ]:
# statistical measures about the data
data_frame.describe()

In [ ]:
# checking the distribution of target variable
data_frame['label'].value_counts()

*1 -> Benign*

*2 -> Malignant*

In [ ]:
data_frame.groupby('label').mean()

**Seperating the features and target**

In [ ]:
X = data_frame.drop(columns='label', axis = 1)
Y = data_frame['label']

In [ ]:
print(X)

In [ ]:
print(Y)

**Splitting the data into training data and testing data**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

**Standardize the data**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

**Building the Neural Network**

In [ ]:
# importing tensorflow and keras
import tensorflow as tf
tf.random.set_seed(3)
from tensorflow import keras

In [ ]:
# setting up the layers of the Neural Network
# Input layer -> Hidden layer -> Output layer
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(30,)),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(2, activation='sigmoid')
    ])

In [ ]:
# compiling the Neural Network
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# training the Neural Network
history = model.fit(X_train_std, Y_train, validation_split=0.1, epochs=10)

**Visualizing accuracy and loss**

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training data', 'validation data'], loc = 'lower right')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training data', 'validation data'], loc = 'upper right')

**Accuracy of the model on test data**

In [ ]:
loss, accuracy = model.evaluate(X_test_std, Y_test)
print(accuracy)

In [ ]:
print(X_test_std.shape)
print(X_test_std[0])

In [ ]:
Y_pred = model.predict(X_test_std)

In [ ]:
print(Y_pred.shape)
print(Y_pred[0])

In [ ]:
print(X_test_std)

In [ ]:
print(Y_pred)

*model.predict() gives the prediction probability of each class for the data point*

In [ ]:
# converting the prediction probability to class labels
Y_pred_labels = [np.argmax(i) for i in Y_pred]
print(Y_pred_labels)

**Building and testing the predictive system**

In [ ]:
model.save('model/BreastCancerNN.h5')

In [ ]:
from keras.models import load_model

model1 = load_model('./model/BreastCancerNN.h5', compile=False)

In [ ]:
def predictTumor():
    input_data_str = input('Please input the data: ')
    input_data_list = input_data_str.split(',')
    input_data_float = [float(value) for value in input_data_list]

    # change the input_data to numpy array
    input_data_as_numpy_array = np.asarray(input_data_float)

    # reshape the numpy array as we are predicting for one data point
    input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

    # standardizing the input data
    input_data_std = scaler.transform(input_data_reshaped)

    prediction = model1.predict(input_data_std)
    print(prediction)

    prediction_label = [np.argmax(prediction)]
    print(prediction_label)

    if(prediction_label[0] == 0):
      print('The tumor is Malignant')

    else:
      print('The tumor is Benign')

In [ ]:
predictTumor()